In [10]:
import os
os.environ['OPENAI_API_KEY'] = ''
os.environ['PINECONE_API_KEY'] = ''
openai_api_key = os.environ['OPENAI_API_KEY']
pinecone_api_key = os.environ['PINECONE_API_KEY']

In [11]:
from datasets import load_dataset
dataset = load_dataset("lcw99/wikipedia-korean-20221001", split='train[:100]')

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,
    separators=["\n\n", "\n", ""," "]
    
)

In [13]:
from pinecone import Pinecone, PodSpec

api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=api_key)

In [15]:
from time import sleep

index_name = 'vectordbprac'
dimension = 1536
metric = 'dotproduct'
spec = PodSpec('gcp-starter')

if index_name in [index_info["name"] for index_info in pc.list_indexes()]:
    pc.delete_index(index_name)

pc.create_index(index_name, dimension=dimension, metric=metric, spec=spec)

while not pc.describe_index(index_name).status['ready']:
    sleep(1)
    
index = pc.Index(index_name)

index_stats = index.describe_index_stats()
print(index_stats)

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


In [18]:
# 청크 단위 임베딩
from langchain.embeddings.openai import OpenAIEmbeddings

model_name = 'text-embedding-3-small'
embed = OpenAIEmbeddings(
    
    model=model_name,
    api_key=openai_api_key
)

In [19]:
from tqdm.auto import tqdm
from uuid import uuid4

def process_and_upload_records(dataset, batch_limit=100):
    texts = []
    metadatas = []
    
    for i, record in enumerate(tqdm(dataset)):
        metadata = {
            'id': str(record['id']),
            'source': record['url'],
            'title': record['title'],
        }
        
        record_texts = text_splitter.split_text(record['text'])
        record_metadatas = [
            {"chunk": j, "text": text, **metadata}
            for j, text in enumerate(record_texts)
        ]
        
        texts.extend(record_texts)
        metadatas.extend(record_metadatas)
        
        if len(texts) >= batch_limit:
            upload_data(texts, metadatas)
            texts, metadatas = [], []
        
    if texts:
        upload_data(texts, metadatas)
  
def upload_data(texts, metadatas):
    ids = [str(uuid4()) for _ in range(len(texts))]
    embeds = embed.embed_documents(texts)
    index.upsert(vectors=zip(ids, embeds, metadatas))
    
process_and_upload_records(dataset)

  0%|          | 0/100 [00:00<?, ?it/s]

In [26]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.01521,
 'namespaces': {'': {'vector_count': 1521}},
 'total_vector_count': 1521}

In [28]:
# pinecone api로 기반 문서들 업서트 이후에 랭체인에서 이를 활용하기 위해 백터스토어 오브젝트로 연결
from langchain_community.vectorstores import Pinecone

text_field = "text"

vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

In [29]:
query = "지미 카터가 누구야?"

vectorstore.similarity_search(
    query, k=3
)


[Document(page_content='제임스 얼 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39대 대통령 (1977년 ~ 1981년)이다.\n\n생애\n\n어린 시절 \n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.', metadata={'chunk': 0.0, 'id': '5', 'source': 'https://ko.wikipedia.org/wiki/%EC%A7%80%EB%AF%B8%20%EC%B9%B4%ED%84%B0', 'title': '지미 카터'}),
 Document(page_content='지미 카터는 대한민국과의 관계에서도 중요한 영향을 미쳤던 대통령 중 하나다. 인권 문제와 주한미군 철수 문제로 한때 한미 관계가 불편하기도 했다. [1978년 대한민국에 대한 북한의 위협에 대비해 한미연합사를 창설하면서, 1982년까지 3단계에 걸쳐 주한미군을 철수하기로 했다. 그러나 주한미군사령부와 정보기관·의회의 반대에 부딪혀 주한미군은 완전철수 대신 6,000명을 감축하는 데 그쳤다. 또한 박정희 정권의 인권 문제 등과의 논란으로 불협화음을 냈으나, 1979년 6월 하순, 대한민국을 방문하여 관계가 다소 회복되었다.', metadata={'chunk': 4.0, 'id': '5', 'source': 'https://ko.wikipedia.org/wiki/%EC%A7%80%EB%AF%B8%20%EC%B9%B4%ED%84%B0', 'title': '지미 카터'}),
 Document(page_content='퇴임 이후 \n\n퇴임 이후 민간 자원을 적극 활용한 비영리 기구인 카터 재단을 설립한 뒤 민주주의 실현을 위해 제 3세계의 선거 감시 활동 및 기니 벌레에 